In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, NoSuchFrameException

In [2]:
import pandas as pd
import numpy as np
import re
import time

In [6]:
def cafe_crawling2():
    description_state = False
    try:
        description_a_link = driver.find_element(By.CSS_SELECTOR, "div.PIbes > div.O8qbU > div.vV_z_ > a > span.zPfVt")
        description_a_link.click()
        description_state = True
    except Exception as ex:
        pass

    # 기본 카페 정보 크롤링
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    address = soup.find('span', class_='LDgIH').text if soup.find('span', class_='LDgIH') else ""
    if not "대구" in address and not "경산" in address:
        return None

    cafe_name = soup.find('span', class_='Fc1rA').text
    cafe_type = soup.find('span', class_='DJJvD').text
    

    image_div = soup.find('div', {'id': 'ibu_1'})
    if image_div:
        style = image_div.get('style')
        background_image_url = re.search(r'url\((.*?)\)', style).group(1)
    else:
        background_image_url = ""
    
    # reviews_div = soup.find('div', class_='dAsGb')
    # starring = float(reviews_div.find('span', class_='PXMot LXIwF').text.strip()[2:]) if reviews_div.find('span', class_='PXMot LXIwF') else 0
    # if not reviews_div.find_all('span', class_='PXMot') or len(reviews_div.find_all('span', class_='PXMot')) < 3:
    #     return None
    # visitor_reviews = int(reviews_div.find_all('span', class_='PXMot')[1 if starring else 0].text.strip().split(" ")[1].replace(",", ""))
    # blog_reviews = int(reviews_div.find_all('span', class_='PXMot')[2 if starring else 1].text.strip().split(" ")[1].replace(",", ""))
    
    try:
        # 별점 요소 찾기
        star_rating_element = driver.find_element(By.CSS_SELECTOR, "div.dAsGb > span.PXMot.LXIwF")
        starring = float(star_rating_element.text.strip()[2:])
    except NoSuchElementException:
        # 별점 요소가 없는 경우
        starring = 0

    try:
        # 방문자 리뷰 요소 찾기
        visitor_review_element = driver.find_element(By.XPATH, "//span[@class='PXMot']/a[contains(@href, '/review/visitor')]")
        visitor_reviews = int(visitor_review_element.text.strip().split(" ")[1].replace(",", ""))
    except NoSuchElementException:
        # 방문자 리뷰 요소가 없는 경우
        visitor_reviews = 0
        
    try:
        # 블로그 리뷰 요소 찾기
        blog_review_element = driver.find_element(By.XPATH, "//span[@class='PXMot']/a[contains(@href, '/review/ugc')]")
        blog_reviews = int(blog_review_element.text.strip().split(" ")[1].replace(",", ""))
    except NoSuchElementException:
        # 블로그 리뷰 요소가 없는 경우
        blog_reviews = 0
    
    if visitor_reviews < 10:
        return None
    #PIbes > O8qbU > vV_z_ > zPfVt
    # description = soup.find_all('span', class_='zPfVt')[1].text if len(soup.find_all('span', class_='zPfVt')) > 1 else ""
    if description_state:
        description = driver.find_element(By.CSS_SELECTOR, "div.PIbes > div.O8qbU > div.vV_z_ > a > span.zPfVt").text
    else:
        try:
            description = driver.find_element(By.CSS_SELECTOR, "div.PIbes > div.O8qbU > div.vV_z_ > div.xHaT3 >  span.zPfVt").text
        except:
            description = ""
    address = soup.find('span', class_='LDgIH').text if soup.find('span', class_='LDgIH') else ""
    business_hours = soup.find('time').text if soup.find('time') else "" 
    phone_number = soup.find('span', class_='xlx7Q').text if soup.find('span', class_='xlx7Q') else ""

    # 키워드로 넘어가기
    keyword_tab_link = driver.find_element(By.XPATH, "//div[@class='flicking-camera']/a[.//span[contains(text(), '리뷰')]]")
    keyword_tab_link.click()
    
    keyword_state = False
    idx = 0
    while not keyword_state and idx < 5:
        try:
            keyword_list_link = WebDriverWait(driver, 0.5).until(EC.presence_of_element_located((By.CLASS_NAME, "Tvx37")))
            keyword_list_link.click()
            keyword_state = True
        except:
            for _ in range(5):
                keyword_tab_link.click()
                time.sleep(0.5)
        idx+=1
    
#     for _ in range(3):
#         try:
#             keyword_list_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Tvx37")))
#             keyword_list_link.click()
#             keyword_state = True
#         except:
#             keyword_tab_link.click()
    
    if not keyword_state:
        return [cafe_name, cafe_type, starring, visitor_reviews, blog_reviews, address, business_hours, phone_number, "", description, "", "", "", background_image_url]

    while True:
        try:
            keyword_more_link = driver.find_element(By.CLASS_NAME, 'Tvx37')
            keyword_more_link.click()
        except Exception as ex:
            break

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    keyword_number = int(driver.find_element(By.CSS_SELECTOR, "div._Wmab > em").text.replace(",", ""))

    keywords = {}
    ui_tags = soup.find('ui', class_='uNsI9')
    li_tags = soup.find_all('li', class_='nbD78')

    for li in li_tags:
        # 첫 번째 숫자를 int 형으로 변환
        text = li.find('span', class_='nWiXa').text.strip('"')
        numbers = re.findall(r'\d+', li.find('span', class_='TwM9q').text)
        number = int(numbers[0]) if numbers else None
        keywords[text] = number
    
    return [cafe_name, cafe_type, starring, visitor_reviews, blog_reviews, address, business_hours, phone_number, "", description, "", keywords, keyword_number, background_image_url]

In [4]:
cafe_search_list= [
                   "동성로", "대구 중앙로", "교동", "경북대병원역",
                    "동대구역", "대구 평화시장", "안심역",
                    "대구 광장코아", "만평역", "대구 서구청",
                    "명덕역", "대구 앞산",
                    "경북대 북문", "대구 복현오거리", "동천동", "칠곡네거리",
                    "범어네거리", "수성유원지", "시지", "지산",
                    "두류공원", "용산", "월배", "계명대"
                    ]

In [7]:
for cafe_name in ["대구 동천동", "칠곡네거리"]:
    try:
        df = pd.DataFrame(columns=['cafe_name', "cafe_type", 'starring', 'visitor_review', 'blog_review', 'address', 'business_hours', 'phone_number', 'connection_site', 'description',
                                    "menu", "keyword", 'keyword_number', 'image_link'])
        # WebDriver 초기화
        driver = webdriver.Chrome() # Chrome 드라이버 사용
        driver.get(f"https://map.naver.com/p/search/{cafe_name} 카페") # 네이버 지도 접속

        # 결과 로드 대기
        time.sleep(3) # 페이지 로드를 위해 잠시 대기


        while True:
            # searchIframe으로 컨텍스트 전환 + div scroll 내리기
            driver.switch_to.frame("searchIframe")
            scroll_div = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "_pcmap_list_scroll_container")))
            for _ in range(5):
                driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scroll_div)
                time.sleep(1)

            # 목록을 담고 있는 ul 태그를 찾고, 그 안의 모든 li 태그를 가져옴
            cafe_list = scroll_div.find_elements(By.TAG_NAME, "li")
            cafe_size = len(cafe_list)


            for item in cafe_list:
                try:
                    place_bluelink = item.find_element(By.CLASS_NAME, "place_bluelink")
                    # 부모의 부모인 a 태그 찾기 및 클릭
                    parent_a_tag = place_bluelink.find_element(By.XPATH, "../..")
                    parent_a_tag.click()

                    time.sleep(3)

                    # 필요한 작업 수행 (예: 상세 정보 수집)
                    driver.switch_to.default_content()
                    driver.switch_to.frame("entryIframe")
                    cafe_row = cafe_crawling2()
                    if cafe_row:
                        df.loc[len(df)] = cafe_row
                        print(cafe_row)

                    # 다음 카페로 이동하기 전에 searchIframe으로 다시 컨텍스트 전
                    driver.switch_to.default_content()
                    driver.switch_to.frame("searchIframe")

                except Exception as e:
                    print(e)
                    pass

            next_a_link = driver.find_elements(By.CLASS_NAME, "eUTV2")[1]
            aria_disabled = next_a_link.get_attribute("aria-disabled")
            # 'aria-disabled'가 'false'이면 a 태그 클릭
            if aria_disabled == "false":
                next_a_link.click()
            else:
                break

            # 원래 프레임으로 돌아감
            driver.switch_to.default_content()
    finally:
            df.to_csv(f"./new_cafe_data/{cafe_name}.csv", encoding='utf-8')


['레이지모닝', '카페,디저트', 4.62, 1534, 604, '대구 북구 동천로 5-15 레이지모닝 칠곡점', '08:00에 영업 시작', '0507-1393-1313', '', '', '', {'디저트가 맛있어요': 604, '커피가 맛있어요': 514, '매장이 청결해요': 274, '음료가 맛있어요': 250, '인테리어가 멋져요': 234, '친절해요': 208, '대화하기 좋아요': 166, '특별한 메뉴가 있어요': 107, '화장실이 깨끗해요': 89, '사진이 잘 나와요': 86, '좌석이 편해요': 71, '가성비가 좋아요': 40, '뷰가 좋아요': 35, '집중하기 좋아요': 33, '주차하기 편해요': 23, '빵이 맛있어요': 8, '매장이 넓어요': 3, '음식이 맛있어요': 1, '아이와 가기 좋아요': 1, '단체모임 하기 좋아요': 1, '종류가 다양해요': 1}, 1081, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20231112_100%2F1699768524175rEhdd_JPEG%2F1699676174230.jpg"']
['만재네 칠곡3지구점', '육류,고기요리', 0, 462, 163, '대구 북구 동천로 125-7 1층', '11:30에 영업 시작', '0507-1364-1928', '', '', '', {'음식이 맛있어요': 392, '고기 질이 좋아요': 351, '매장이 넓어요': 238, '직접 잘 구워줘요': 190, '친절해요': 184, '인테리어가 멋져요': 99, '양이 많아요': 93, '매장이 청결해요': 74, '단체모임 하기 좋아요': 62, '특별한 메뉴가 있어요': 53, '가성비가 좋아요': 23, '반찬이 잘 나와요': 11, '특별한 날 가기 좋아요': 8, '주차하기 편해요': 7, '재료가 신선해요': 4, '잡내가 적어요

['메가MGC커피 칠곡3지구점', '카페', 4.61, 507, 21, '대구 북구 동천로 133', '09:00에 영업 시작', '053-322-6088', '', '', '', {'커피가 맛있어요': 205, '가성비가 좋아요': 138, '음료가 맛있어요': 128, '친절해요': 94, '디저트가 맛있어요': 46, '매장이 청결해요': 42, '특별한 메뉴가 있어요': 26, '대화하기 좋아요': 14, '집중하기 좋아요': 9, '인테리어가 멋져요': 8, '좌석이 편해요': 5, '사진이 잘 나와요': 5, '화장실이 깨끗해요': 5, '뷰가 좋아요': 2, '양이 많아요': 1, '주차하기 편해요': 1}, 378, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20190128_170%2F1548676559178TrvFY_JPEG%2FU0VVk7UwvSnYJf9q9nOGXLWZ.jpeg.jpg"']
['미니미', '베이커리', 0, 342, 20, '대구 북구 동천로 41 1층 미니미', '09:00에 영업 시작', '0507-1403-7254', '', '', '', {'빵이 맛있어요': 307, '친절해요': 265, '커피가 맛있어요': 167, '매장이 청결해요': 163, '특별한 메뉴가 있어요': 150, '음료가 맛있어요': 114, '가성비가 좋아요': 99, '인테리어가 멋져요': 71, '디저트가 맛있어요': 24, '사진이 잘 나와요': 19, '대화하기 좋아요': 16, '메뉴 구성이 알차요': 9, '건강한 맛이에요': 8, '아늑해요': 3, '주차하기 편해요': 3, '종류가 다양해요': 3, '집중하기 좋아요': 2, '뷰가 좋아요': 2, '좌석이 편해요': 1, '음식이 맛있어요': 1, '재료가 신선해요': 1, '화장실이 깨끗해요': 1}, 341, '"ht

['디저트39 대구동천점', '카페', 4.57, 803, 72, '대구 북구 동천로 128-6 디져트39 대구동천점', '10:00에 영업 시작', '053-325-8742', '', '', '', {'디저트가 맛있어요': 291, '음료가 맛있어요': 268, '커피가 맛있어요': 191, '가성비가 좋아요': 133, '매장이 청결해요': 130, '친절해요': 99, '인테리어가 멋져요': 83, '특별한 메뉴가 있어요': 80, '좌석이 편해요': 69, '대화하기 좋아요': 64, '사진이 잘 나와요': 22, '화장실이 깨끗해요': 22, '집중하기 좋아요': 12, '뷰가 좋아요': 10, '주차하기 편해요': 5, '매장이 넓어요': 4, '차분한 분위기에요': 1, '양이 많아요': 1, '메뉴 구성이 알차요': 1, '건강한 맛이에요': 1, '종류가 다양해요': 1}, 717, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20211007_90%2F1633590290017Ci76G_JPEG%2F010KnMiSChWMuOE4UBaQxJn2.jpg"']
['미유버터', '카페,디저트', 0, 69, 48, '대구 북구 동천로 67 1 층', '09:00에 영업 시작', '0507-1334-1066', '', '', '', {'디저트가 맛있어요': 53, '친절해요': 35, '매장이 청결해요': 27, '커피가 맛있어요': 23, '특별한 메뉴가 있어요': 19, '인테리어가 멋져요': 16, '사진이 잘 나와요': 11, '음료가 맛있어요': 10, '가성비가 좋아요': 8, '대화하기 좋아요': 5, '좌석이 편해요': 3, '화장실이 깨끗해요': 3, '주차하기 편해요': 1, '집중하기 좋아요': 1, '뷰가 좋아요': 1}, 68, '"https://search.pstatic.ne

['더벤티 대구칠곡점', '카페', 4.38, 423, 13, '대구 북구 학정로 421 1층, 더벤티 대구칠곡점', '08:00에 영업 시작', '053-323-8656', '', '', '', {'가성비가 좋아요': 77, '커피가 맛있어요': 72, '음료가 맛있어요': 51, '친절해요': 45, '매장이 청결해요': 16, '디저트가 맛있어요': 15, '특별한 메뉴가 있어요': 12, '좌석이 편해요': 2, '뷰가 좋아요': 2, '인테리어가 멋져요': 1, '양이 많아요': 1, '주차하기 편해요': 1, '사진이 잘 나와요': 1, '집중하기 좋아요': 1, '대화하기 좋아요': 1, '차가 맛있어요': 1}, 203, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20200617_94%2F1592362308610bnDTF_JPEG%2FguxhSS81pQo8vNapEfQwT65_.jpg"']
['투썸플레이스 대구칠곡3지구점', '카페', 4.48, 1215, 52, '대구 북구 동암로12길 15 1층', '08:00에 영업 시작', '0507-1494-3457', '', '', '', {'커피가 맛있어요': 437, '디저트가 맛있어요': 290, '음료가 맛있어요': 212, '매장이 청결해요': 209, '대화하기 좋아요': 189, '친절해요': 188, '좌석이 편해요': 147, '인테리어가 멋져요': 134, '집중하기 좋아요': 56, '특별한 메뉴가 있어요': 48, '가성비가 좋아요': 40, '주차하기 편해요': 30, '사진이 잘 나와요': 24, '화장실이 깨끗해요': 21, '뷰가 좋아요': 14, '매장이 넓어요': 3, '음식이 맛있어요': 1, '양이 많아요': 1}, 944, '']
['아마스빈 대구칠곡점', '카페', 4.52, 1401, 26, '대구 북구 

['커피상점', '카페,디저트', 4.49, 73, 13, '대구 북구 동천로24길 5', '10:00에 영업 시작', '070-4189-1391', '', '', '', {'커피가 맛있어요': 26, '음료가 맛있어요': 8, '친절해요': 7, '가성비가 좋아요': 5, '주차하기 편해요': 4, '디저트가 맛있어요': 2, '인테리어가 멋져요': 2, '특별한 메뉴가 있어요': 2, '매장이 청결해요': 2, '좌석이 편해요': 1, '대화하기 좋아요': 1}, 37, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20181110_235%2F1541822851805zVaUS_JPEG%2FOF3ilu22mJOeZ7rcWnOV9vc-.jpeg.jpg"']
['카페봄봄 칠곡점', '카페', 4.29, 405, 8, '대구 북구 학정로 439', '10:00에 영업 시작', '0507-0289-9150', '', '', '', {'커피가 맛있어요': 122, '가성비가 좋아요': 75, '음료가 맛있어요': 66, '친절해요': 42, '디저트가 맛있어요': 19, '인테리어가 멋져요': 8, '매장이 청결해요': 8, '특별한 메뉴가 있어요': 7, '대화하기 좋아요': 7, '집중하기 좋아요': 3, '뷰가 좋아요': 3, '사진이 잘 나와요': 2, '주차하기 편해요': 1, '화장실이 깨끗해요': 1}, 247, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20200128_184%2F15802006867398gVdJ_JPEG%2FGE_ldwriDVcdr7YkbSN0yLRI.jpg"']
['프랑제리 칠곡홈플러스점', '베이커리', 4.73, 1554

...

['카페플로리안', '카페', 4.19, 161, 0, '대구 북구 구암로39길 1', '정보없음', '053-311-3937', '', '', '', {'커피가 맛있어요': 48, '친절해요': 37, '음료가 맛있어요': 19, '대화하기 좋아요': 19, '가성비가 좋아요': 18, '매장이 청결해요': 15, '디저트가 맛있어요': 12, '좌석이 편해요': 6, '특별한 메뉴가 있어요': 6, '인테리어가 멋져요': 5, '집중하기 좋아요': 5, '화장실이 깨끗해요': 4, '뷰가 좋아요': 2, '주차하기 편해요': 1, '사진이 잘 나와요': 1}, 111, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220922_240%2F1663817481087Jhyya_JPEG%2F83D9F25E-0AF8-4882-A6D3-B0605227A2C6.jpeg"']
['빈스크라운커피', '테이크아웃커피', 4.48, 167, 35, '대구 북구 팔거천동로 218 1, 2층', '11:00에 영업 시작', '053-313-3222', '', '', '', {'커피가 맛있어요': 51, '디저트가 맛있어요': 26, '친절해요': 25, '대화하기 좋아요': 24, '음료가 맛있어요': 20, '집중하기 좋아요': 16, '매장이 청결해요': 13, '좌석이 편해요': 11, '가성비가 좋아요': 11, '인테리어가 멋져요': 8, '특별한 메뉴가 있어요': 8, '뷰가 좋아요': 6, '주차하기 편해요': 2, '화장실이 깨끗해요': 2, '컨셉이 독특해요': 1, '매장이 넓어요': 1}, 111, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F201

['베르나르 스터디카페', '스터디카페', 4.73, 120, 19, '대구 북구 동천로 126 6층', '연중무휴', '010-5659-1571', '', '', '', {'요금제가 잘 되어있어요': 14, '집중하기 좋아요': 10, '좌석이 편해요': 7, '커피가 맛있어요': 6, '가성비가 좋아요': 6, '매장이 청결해요': 6, '화장실이 깨끗해요': 6, '룸이 잘 되어있어요': 5, '분위기 관리가 철저해요': 4, '매장이 넓어요': 4, '인테리어가 멋져요': 3, '친절해요': 2, '차분한 분위기에요': 1, '비싼 만큼 가치있어요': 1}, 30, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20201217_122%2F1608182336537J5b0r_PNG%2FEj9qKoqHj9EfPcw4ML8Po04J.png"']
['카페 봄봄 칠곡운암역점', '카페', 4.32, 331, 3, '대구 북구 구암로 137', '10:00에 영업 시작', '0507-0289-9151', '', '', '', {'커피가 맛있어요': 67, '음료가 맛있어요': 46, '가성비가 좋아요': 45, '친절해요': 36, '디저트가 맛있어요': 12, '매장이 청결해요': 7, '특별한 메뉴가 있어요': 5, '뷰가 좋아요': 2, '인테리어가 멋져요': 1, '사진이 잘 나와요': 1, '집중하기 좋아요': 1, '화장실이 깨끗해요': 1}, 144, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20200806_276%2F15966903401229AyzM_JPEG%2FC4r3XStLxtIl3CILpUlXFunu.jpeg.jpg"']
['포인트커피

['몬스터커피 팔거역점', '카페', 4.45, 751, 23, '대구 북구 팔거천동로 217', '08:00에 영업 시작', '053-313-3683', '', '', '', {'커피가 맛있어요': 263, '음료가 맛있어요': 176, '가성비가 좋아요': 148, '친절해요': 87, '매장이 청결해요': 58, '디저트가 맛있어요': 37, '특별한 메뉴가 있어요': 25, '대화하기 좋아요': 24, '인테리어가 멋져요': 15, '좌석이 편해요': 8, '뷰가 좋아요': 7, '사진이 잘 나와요': 5, '집중하기 좋아요': 5, '화장실이 깨끗해요': 5, '주차하기 편해요': 2, '음식이 빨리 나와요': 1, '메뉴 구성이 알차요': 1}, 484, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220216_187%2F1644999977122Be7AG_JPEG%2FAA3C4983-700C-43E8-B1AD-10B555F23BE5.jpeg"']
['메가MGC커피 대구태전점', '카페', 4.49, 262, 5, '대구 북구 칠곡중앙대로 254 1층', '10:00에 영업 시작', '070-7543-8604', '', '', '', {'커피가 맛있어요': 164, '가성비가 좋아요': 115, '친절해요': 102, '음료가 맛있어요': 95, '매장이 청결해요': 17, '디저트가 맛있어요': 16, '특별한 메뉴가 있어요': 6, '대화하기 좋아요': 5, '인테리어가 멋져요': 4, '양이 많아요': 4, '메뉴 구성이 알차요': 3, '주차하기 편해요': 3, '뷰가 좋아요': 3, '좌석이 편해요': 2, '아늑해요': 2, '집중하기 좋아요': 2, '컨셉이 독특해요': 1, '음식이 맛있어요': 1, '사진이 잘 나와요': 1}, 216, '"https://search.pstat

['청솔로9 칠곡동천점', '카페,디저트', 0, 179, 14, '대구 북구 동암로12길 22 101호', '09:00에 영업 시작', '0507-872-3865', '', '', '', {'커피가 맛있어요': 84, '음료가 맛있어요': 60, '친절해요': 47, '가성비가 좋아요': 47, '매장이 청결해요': 27, '디저트가 맛있어요': 14, '특별한 메뉴가 있어요': 11, '대화하기 좋아요': 6, '인테리어가 멋져요': 5, '좌석이 편해요': 4, '사진이 잘 나와요': 2, '집중하기 좋아요': 2, '뷰가 좋아요': 2, '주차하기 편해요': 1, '화장실이 깨끗해요': 1}, 176, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220830_216%2F1661844683425A5gC0_PNG%2Fimage0_%25281%2529.png"']
['티타임커피 아이스크림스토리 동천점', '아이스크림', 0, 68, 3, '대구 북구 팔거천동로 104 205동 104호', '연중무휴', '010-2445-3699', '', '', '', {'매장이 청결해요': 37, '가성비가 좋아요': 23, '커피가 맛있어요': 19, '음료가 맛있어요': 16, '친절해요': 13, '디저트가 맛있어요': 10, '인테리어가 멋져요': 8, '대화하기 좋아요': 8, '좌석이 편해요': 5, '사진이 잘 나와요': 4, '집중하기 좋아요': 4, '화장실이 깨끗해요': 2, '특별한 메뉴가 있어요': 1}, 67, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20211130_45%2F1638257348879M0E0L_JPEG%2FKakaoT

['에이치유아이', '카페,디저트', 0, 122, 2, '대구 북구 구리로 25 1층', '10:00에 영업 시작', '053-325-8424', '', '', '', {'커피가 맛있어요': 85, '디저트가 맛있어요': 61, '인테리어가 멋져요': 61, '친절해요': 48, '음료가 맛있어요': 35, '매장이 청결해요': 33, '주차하기 편해요': 26, '대화하기 좋아요': 22, '사진이 잘 나와요': 13, '화장실이 깨끗해요': 13, '특별한 메뉴가 있어요': 11, '좌석이 편해요': 8, '가성비가 좋아요': 6, '뷰가 좋아요': 5, '매장이 넓어요': 4, '집중하기 좋아요': 4, '음악이 좋아요': 2, '오래 머무르기 좋아요': 2, '아늑해요': 1, '음식이 맛있어요': 1, '아이와 가기 좋아요': 1, '단체모임 하기 좋아요': 1}, 104, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20231113_155%2F1699860860252WonaG_JPEG%2F1000008693.jpg"']
['메이플빵', '베이커리', 4.54, 395, 16, '대구 북구 태전로 1', '', '053-311-4886', '', '', '', {'빵이 맛있어요': 195, '친절해요': 82, '매장이 청결해요': 40, '특별한 메뉴가 있어요': 36, '커피가 맛있어요': 26, '가성비가 좋아요': 19, '음료가 맛있어요': 14, '주차하기 편해요': 8, '인테리어가 멋져요': 4, '차분한 분위기에요': 3, '대화하기 좋아요': 3, '음식이 빨리 나와요': 2, '화장실이 깨끗해요': 2, '재료가 신선해요': 1, '종류가 다양해요': 1, '뷰가 좋아요': 1}, 233, '"https://search.pstatic.net/common/?autoRotate=